# Class For DecisionTree

In [8]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


class CardioDecisionTreeClassifier:
    """
    A reusable Decision Tree classifier for cardiovascular disease prediction.
    
    Features:
    - Train/test split with customizable ratio
    - Optional feature scaling
    - Comprehensive performance metrics
    - Export predictions to CSV
    """
    
    def __init__(self, test_size=0.2, random_state=42, max_depth=None,
                 min_samples_split=2, min_samples_leaf=1, criterion='gini',
                 class_weight='balanced', scale_features=True,
                 target_col='cardio', drop_cols=None):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        test_size : float, default=0.2
            Proportion of dataset to include in test split
        random_state : int, default=42
            Random state for reproducibility
        max_depth : int, default=None
            Maximum depth of the tree
        min_samples_split : int, default=2
            Minimum samples required to split a node
        min_samples_leaf : int, default=1
            Minimum samples required at a leaf node
        criterion : str, default='gini'
            The function to measure the quality of a split
        class_weight : str or dict, default='balanced'
            Weights associated with classes
        scale_features : bool, default=True
            Whether to scale features
        target_col : str, default='cardio'
            Name of the target variable column
        drop_cols : list, default=None
            List of columns to drop (e.g., ID columns)
        """
        
        self.test_size = test_size
        self.random_state = random_state
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.class_weight = class_weight
        self.scale_features = scale_features
        self.target_col = target_col
        self.drop_cols = drop_cols if drop_cols else []
        
        self.model = None
        self.scaler = None
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None
        self.metrics = {}
        
    def load_data(self, filepath):
        """
        Load data from CSV file.
        
        Parameters:
        -----------
        filepath : str
            Path to the CSV file
            
        Returns:
        --------
        pd.DataFrame : Loaded dataframe
        """
        self.data = pd.read_csv(filepath)
        print(f"Data loaded successfully: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
        return self.data
    
    def preprocess_data(self):
        """
        Preprocess the data for training.
        Uses `self.target_col` and `self.drop_cols` from initialization.
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
        
        df = self.data.copy()
        if self.drop_cols:
            df = df.drop(columns=self.drop_cols, errors='ignore')
        
        X = df.drop(columns=[self.target_col])
        y = df[self.target_col]
        
        self.feature_names = X.columns.tolist()
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        if self.scale_features:
            self.scaler = StandardScaler()
            self.X_train = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                columns=self.feature_names,
                index=self.X_train.index
            )
            self.X_test = pd.DataFrame(
                self.scaler.transform(self.X_test),
                columns=self.feature_names,
                index=self.X_test.index
            )
        
        print(f"Data preprocessed: Train size = {len(self.X_train)}, Test size = {len(self.X_test)}")
    
    def train(self):
        """Train the Decision Tree classifier."""
        if self.X_train is None:
            raise ValueError("Data not preprocessed. Call preprocess_data() first.")
        
        self.model = DecisionTreeClassifier(
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            criterion=self.criterion,
            class_weight=self.class_weight,
            random_state=self.random_state
        )
        
        self.model.fit(self.X_train, self.y_train)
        print("Model trained successfully!")
    
    def predict(self, X=None):
        """
        Make predictions on data.
        
        Parameters:
        -----------
        X : pd.DataFrame, default=None
            Data to predict. If None, uses test set.
            
        Returns:
        --------
        np.ndarray : Predictions
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        if X is None:
            X_to_predict = self.X_test
        else:
            X_to_predict = X

        if X is not None and self.scale_features and self.scaler is not None:
            X_to_predict = pd.DataFrame(
                self.scaler.transform(X_to_predict),
                columns=X_to_predict.columns,
                index=X_to_predict.index
            )
        
        return self.model.predict(X_to_predict)
    
    def evaluate(self):
        """
        Evaluate model performance on test set.
        
        Returns:
        --------
        dict : Dictionary containing all performance metrics
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        y_pred = self.predict()
        
        self.metrics = {
            'accuracy': accuracy_score(self.y_test, y_pred),
            'precision': precision_score(self.y_test, y_pred, average='binary'),
            'recall': recall_score(self.y_test, y_pred, average='binary'),
            'f1_score': f1_score(self.y_test, y_pred, average='binary'),
            'confusion_matrix': confusion_matrix(self.y_test, y_pred)
        }
        
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print(f"Accuracy:  {self.metrics['accuracy']:.4f}")
        print(f"Precision: {self.metrics['precision']:.4f}")
        print(f"Recall:    {self.metrics['recall']:.4f}")
        print(f"F1-Score:  {self.metrics['f1_score']:.4f}")
        print("\nConfusion Matrix:")
        print(self.metrics['confusion_matrix'])
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, 
                                      target_names=['No Disease', 'Disease']))
        print("="*50 + "\n")
        
        return self.metrics
    
    def export_predictions(self, output_filepath, include_all_data=True):
        """
        Export predictions for the *entire* loaded dataset to a new CSV file.
        
        Parameters:
        -----------
        output_filepath : str
            Path for the output CSV file
        include_all_data : bool, default=True
            If True, includes all original columns. If False, only includes
            the target and predicted columns.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
            
        
        # Prepare the full dataset for prediction
        X_full = self.data.copy()
        
        if self.drop_cols:
            X_full = X_full.drop(columns=self.drop_cols, errors='ignore')
            
        X_full = X_full.drop(columns=[self.target_col], errors='ignore')
        
        # Ensure columns match training data
        X_full = X_full[self.feature_names]
        
        if self.scale_features and self.scaler is not None:
            X_full_scaled = pd.DataFrame(
                self.scaler.transform(X_full),
                columns=self.feature_names,
                index=X_full.index
            )
            predictions = self.model.predict(X_full_scaled)
        else:
            predictions = self.model.predict(X_full)
        
        if include_all_data:
            output_df = self.data.copy()
        else:
            output_df = pd.DataFrame()
            output_df[self.target_col] = self.data[self.target_col]
        
        output_df['predicted'] = predictions
        
        output_df.to_csv(output_filepath, index=False)
        print(f"Predictions exported to: {output_filepath}")
        
        return output_df
    
    def run_full_pipeline(self, input_filepath, output_filepath, include_all_data=True):
        """
        Run the complete pipeline: load, preprocess, train, evaluate, and export.
        
        Parameters:
        -----------
        input_filepath : str
            Path to input CSV file
        output_filepath : str
            Path for output CSV file with predictions
        include_all_data : bool, default=True
            Passed to export_predictions to control output file content.
            
        Returns:
        --------
        dict : Performance metrics
        """
        print("Starting full pipeline...\n")
        
        self.load_data(input_filepath)
        self.preprocess_data()
        self.train()
        metrics = self.evaluate()
        self.export_predictions(output_filepath, include_all_data=include_all_data)
        
        print("Pipeline completed successfully!")
        return metrics


# Example usage
if __name__ == "__main__":
    # Configure the model with all parameters during initialization
    clf = CardioDecisionTreeClassifier(
        test_size=0.2,
        random_state=42,
        max_depth=15,
        min_samples_split=50,
        min_samples_leaf=20,
        criterion='gini',
        class_weight='balanced',
        scale_features=True,
        target_col='cardio',  # <--- Parameter moved here
        drop_cols=['id']      # <--- Parameter moved here
    )
    
    # Run the pipeline with just the filepaths
    matrix_decisiontree = clf.run_full_pipeline(
        input_filepath='train.csv',
        output_filepath='cardio_train_pred.csv'
    )


Starting full pipeline...

Data loaded successfully: 68520 rows, 13 columns
Data preprocessed: Train size = 54816, Test size = 13704
Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7183
Precision: 0.7245
Recall:    0.6942
F1-Score:  0.7090

Confusion Matrix:
[[5141 1788]
 [2072 4703]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.71      0.74      0.73      6929
     Disease       0.72      0.69      0.71      6775

    accuracy                           0.72     13704
   macro avg       0.72      0.72      0.72     13704
weighted avg       0.72      0.72      0.72     13704


Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7183
Precision: 0.7245
Recall:    0.6942
F1-Score:  0.7090

Confusion Matrix:
[[5141 1788]
 [2072 4703]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.71      0.74      0.73      6929
     Disease       0.72      0.69   

# Logistic Regression


In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


class CardioLogisticRegressionClassifier:
    """
    A reusable Logistic Regression classifier for cardiovascular disease prediction.
    
    Features:
    - Train/test split with customizable ratio
    - Optional feature scaling
    - Comprehensive performance metrics
    - Export predictions to CSV
    """
    
    def __init__(self, test_size=0.2, random_state=42, C=1.0, 
                 solver='lbfgs', max_iter=1000, penalty='l2',
                 class_weight='balanced', scale_features=True,
                 target_col='cardio', drop_cols=None):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        test_size : float, default=0.2
            Proportion of dataset to include in test split
        random_state : int, default=42
            Random state for reproducibility
        C : float, default=1.0
            Inverse of regularization strength; smaller values specify stronger regularization
        solver : str, default='lbfgs'
            Algorithm to use in the optimization problem
        max_iter : int, default=1000
            Maximum number of iterations taken for the solvers to converge
        penalty : str, default='l2'
            Specify the norm of the penalty
        class_weight : str or dict, default='balanced'
            Weights associated with classes
        scale_features : bool, default=True
            Whether to scale features
        target_col : str, default='cardio'
            Name of the target variable column
        drop_cols : list, default=None
            List of columns to drop (e.g., ID columns)
        """

        self.test_size = test_size
        self.random_state = random_state
        self.C = C
        self.solver = solver
        self.max_iter = max_iter
        self.penalty = penalty
        self.class_weight = class_weight
        self.scale_features = scale_features
        self.target_col = target_col
        self.drop_cols = drop_cols if drop_cols else []
        
        self.model = None
        self.scaler = None
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None
        self.metrics = {}
        
    def load_data(self, filepath):
        """
        Load data from CSV file.
        
        Parameters:
        -----------
        filepath : str
            Path to the CSV file
            
        Returns:
        --------
        pd.DataFrame : Loaded dataframe
        """
        self.data = pd.read_csv(filepath)
        print(f"Data loaded successfully: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
        return self.data
    
    def preprocess_data(self):
        """
        Preprocess the data for training.
        Uses `self.target_col` and `self.drop_cols` from initialization.
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
        
        df = self.data.copy()
        if self.drop_cols:
            df = df.drop(columns=self.drop_cols, errors='ignore')
        
        X = df.drop(columns=[self.target_col])
        y = df[self.target_col]
        
        self.feature_names = X.columns.tolist()
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        if self.scale_features:
            self.scaler = StandardScaler()
            self.X_train = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                columns=self.feature_names,
                index=self.X_train.index
            )
            self.X_test = pd.DataFrame(
                self.scaler.transform(self.X_test),
                columns=self.feature_names,
                index=self.X_test.index
            )
        
        print(f"Data preprocessed: Train size = {len(self.X_train)}, Test size = {len(self.X_test)}")
    
    def train(self):
        """Train the Logistic Regression classifier."""
        if self.X_train is None:
            raise ValueError("Data not preprocessed. Call preprocess_data() first.")
        
        self.model = LogisticRegression(
            C=self.C,
            solver=self.solver,
            max_iter=self.max_iter,
            penalty=self.penalty,
            class_weight=self.class_weight,
            random_state=self.random_state
        )
        
        self.model.fit(self.X_train, self.y_train)
        print("Model trained successfully!")
    
    def predict(self, X=None):
        """
        Make predictions on data.
        
        Parameters:
        -----------
        X : pd.DataFrame, default=None
            Data to predict. If None, uses test set.
            
        Returns:
        --------
        np.ndarray : Predictions
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        if X is None:
            X_to_predict = self.X_test
        else:
            X_to_predict = X

        if X is not None and self.scale_features and self.scaler is not None:
            # When predicting on new, external data
            X_to_predict = pd.DataFrame(
                self.scaler.transform(X_to_predict),
                columns=X_to_predict.columns,
                index=X_to_predict.index
            )
        
        return self.model.predict(X_to_predict)
    
    def evaluate(self):
        """
        Evaluate model performance on test set.
        
        Returns:
        --------
        dict : Dictionary containing all performance metrics
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        y_pred = self.predict()
        
        self.metrics = {
            'accuracy': accuracy_score(self.y_test, y_pred),
            'precision': precision_score(self.y_test, y_pred, average='binary'),
            'recall': recall_score(self.y_test, y_pred, average='binary'),
            'f1_score': f1_score(self.y_test, y_pred, average='binary'),
            'confusion_matrix': confusion_matrix(self.y_test, y_pred)
        }
        
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print(f"Accuracy:  {self.metrics['accuracy']:.4f}")
        print(f"Precision: {self.metrics['precision']:.4f}")
        print(f"Recall:    {self.metrics['recall']:.4f}")
        print(f"F1-Score:  {self.metrics['f1_score']:.4f}")
        print("\nConfusion Matrix:")
        print(self.metrics['confusion_matrix'])
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, 
                                      target_names=['No Disease', 'Disease']))
        print("="*50 + "\n")
        
        return self.metrics
    
    def export_predictions(self, output_filepath, include_all_data=True):
        """
        Export predictions for the *entire* loaded dataset to a new CSV file.
        
        Parameters:
        -----------
        output_filepath : str
            Path for the output CSV file
        include_all_data : bool, default=True
            If True, includes all original columns. If False, only includes
            the target and predicted columns.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
            
        
        # Prepare the full dataset for prediction
        X_full = self.data.copy()
        
        if self.drop_cols:
            X_full = X_full.drop(columns=self.drop_cols, errors='ignore')
            
        X_full = X_full.drop(columns=[self.target_col], errors='ignore')
        
        # Ensure columns match training data
        X_full = X_full[self.feature_names]
        
        if self.scale_features and self.scaler is not None:
            X_full_scaled = pd.DataFrame(
                self.scaler.transform(X_full),
                columns=self.feature_names,
                index=X_full.index
            )
            predictions = self.model.predict(X_full_scaled)
        else:
            predictions = self.model.predict(X_full)
        
        if include_all_data:
            output_df = self.data.copy()
        else:
            output_df = pd.DataFrame()
            output_df[self.target_col] = self.data[self.target_col]
        
        output_df['predicted'] = predictions
        
        output_df.to_csv(output_filepath, index=False)
        print(f"Predictions exported to: {output_filepath}")
        
        return output_df
    
    def run_full_pipeline(self, input_filepath, output_filepath, include_all_data=True):
        """
        Run the complete pipeline: load, preprocess, train, evaluate, and export.
        
        Parameters:
        -----------
        input_filepath : str
            Path to input CSV file
        output_filepath : str
            Path for output CSV file with predictions
        include_all_data : bool, default=True
            Passed to export_predictions to control output file content.
            
        Returns:
        --------
        dict : Performance metrics
        """
        print("Starting full pipeline...\n")
        
        self.load_data(input_filepath)
        self.preprocess_data()
        self.train()
        metrics = self.evaluate()
        self.export_predictions(output_filepath, include_all_data=include_all_data)
        
        print("Pipeline completed successfully!")
        return metrics


# Example usage
if __name__ == "__main__":
    # Configure the model with all parameters during initialization
    clf = CardioLogisticRegressionClassifier(
        test_size=0.2,
        random_state=42,
        C=0.1,
        solver='lbfgs',
        max_iter=10000, # Increased max_iter for convergence
        penalty='l2',
        class_weight='balanced',
        scale_features=True,
        target_col='cardio',  # <--- Parameter moved here
        drop_cols=['id']      # <--- Parameter moved here
    )
    
    # Run the pipeline with just the filepaths
    matrix_logisticregression = clf.run_full_pipeline(
        input_filepath='train.csv',
        output_filepath='Prediction_LogisticRegression.csv' # Changed output name
    )
    print(matrix_logisticregression)


Starting full pipeline...

Data loaded successfully: 68520 rows, 13 columns
Data preprocessed: Train size = 54816, Test size = 13704
Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7274
Precision: 0.7504
Recall:    0.6722
F1-Score:  0.7091

Confusion Matrix:
[[5414 1515]
 [2221 4554]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.71      0.78      0.74      6929
     Disease       0.75      0.67      0.71      6775

    accuracy                           0.73     13704
   macro avg       0.73      0.73      0.73     13704
weighted avg       0.73      0.73      0.73     13704


Predictions exported to: Prediction_LogisticRegression.csv
Pipeline completed successfully!
{'accuracy': 0.7273788674839463, 'precision': 0.750370736529906, 'recall': 0.6721771217712177, 'f1_score': 0.7091248832139521, 'confusion_matrix': array([[5414, 1515],
       [2221, 4554]])}
Predictions exported to: Prediction_LogisticRegression.csv


# Random Forest


In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


class CardioRandomForestClassifier:
    """
    A reusable Random Forest classifier for cardiovascular disease prediction.
    
    Features:
    - Train/test split with customizable ratio
    - Optional feature scaling
    - Comprehensive performance metrics
    - Export predictions to CSV
    """
    
    def __init__(self, test_size=0.2, random_state=42, n_estimators=100, 
                 max_depth=None, min_samples_split=2, min_samples_leaf=1,
                 max_features='sqrt', class_weight='balanced',
                 scale_features=True, target_col='cardio', drop_cols=None):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        test_size : float, default=0.2
            Proportion of dataset to include in test split
        random_state : int, default=42
            Random state for reproducibility
        n_estimators : int, default=100
            Number of trees in the forest
        max_depth : int, default=None
            Maximum depth of the tree
        min_samples_split : int, default=2
            Minimum samples required to split a node
        min_samples_leaf : int, default=1
            Minimum samples required at a leaf node
        max_features : str or int, default='sqrt'
            Number of features to consider when looking for the best split
        class_weight : str or dict, default='balanced'
            Weights associated with classes
        scale_features : bool, default=True
            Whether to scale features
        target_col : str, default='cardio'
            Name of the target variable column
        drop_cols : list, default=None
            List of columns to drop (e.g., ID columns)
        """

        self.test_size = test_size
        self.random_state = random_state
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.class_weight = class_weight
        self.scale_features = scale_features
        self.target_col = target_col
        self.drop_cols = drop_cols if drop_cols else []
        
        self.model = None
        self.scaler = None
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None
        self.metrics = {}
        
    def load_data(self, filepath):
        """
        Load data from CSV file.
        
        Parameters:
        -----------
        filepath : str
            Path to the CSV file
            
        Returns:
        --------
        pd.DataFrame : Loaded dataframe
        """
        self.data = pd.read_csv(filepath)
        print(f"Data loaded successfully: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
        return self.data
    
    def preprocess_data(self):
        """
        Preprocess the data for training.
        Uses `self.target_col` and `self.drop_cols` from initialization.
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
        
        df = self.data.copy()
        if self.drop_cols:
            df = df.drop(columns=self.drop_cols, errors='ignore')
        
        X = df.drop(columns=[self.target_col])
        y = df[self.target_col]
        
        self.feature_names = X.columns.tolist()
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        if self.scale_features:
            self.scaler = StandardScaler()
            self.X_train = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                columns=self.feature_names,
                index=self.X_train.index
            )
            self.X_test = pd.DataFrame(
                self.scaler.transform(self.X_test),
                columns=self.feature_names,
                index=self.X_test.index
            )
        
        print(f"Data preprocessed: Train size = {len(self.X_train)}, Test size = {len(self.X_test)}")
    
    def train(self):
        """Train the Random Forest classifier."""
        if self.X_train is None:
            raise ValueError("Data not preprocessed. Call preprocess_data() first.")
        
        self.model = RandomForestClassifier(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            max_features=self.max_features,
            class_weight=self.class_weight,
            random_state=self.random_state,
            n_jobs=-1
        )
        
        self.model.fit(self.X_train, self.y_train)
        print("Model trained successfully!")
    
    def predict(self, X=None):
        """
        Make predictions on data.
        
        Parameters:
        -----------
        X : pd.DataFrame, default=None
            Data to predict. If None, uses test set.
            
        Returns:
        --------
        np.ndarray : Predictions
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        if X is None:
            X_to_predict = self.X_test
        else:
            X_to_predict = X

        if X is not None and self.scale_features and self.scaler is not None:
            # When predicting on new, external data
            X_to_predict = pd.DataFrame(
                self.scaler.transform(X_to_predict),
                columns=X_to_predict.columns,
                index=X_to_predict.index
            )
        
        return self.model.predict(X_to_predict)
    
    def evaluate(self):
        """
        Evaluate model performance on test set.
        
        Returns:
        --------
        dict : Dictionary containing all performance metrics
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        y_pred = self.predict()
        
        self.metrics = {
            'accuracy': accuracy_score(self.y_test, y_pred),
            'precision': precision_score(self.y_test, y_pred, average='binary'),
            'recall': recall_score(self.y_test, y_pred, average='binary'),
            'f1_score': f1_score(self.y_test, y_pred, average='binary'),
            'confusion_matrix': confusion_matrix(self.y_test, y_pred)
        }
        
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print(f"Accuracy:  {self.metrics['accuracy']:.4f}")
        print(f"Precision: {self.metrics['precision']:.4f}")
        print(f"Recall:    {self.metrics['recall']:.4f}")
        print(f"F1-Score:  {self.metrics['f1_score']:.4f}")
        print("\nConfusion Matrix:")
        print(self.metrics['confusion_matrix'])
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, 
                                      target_names=['No Disease', 'Disease']))
        print("="*50 + "\n")
        
        return self.metrics
    
    def export_predictions(self, output_filepath, include_all_data=True):
        """
        Export predictions for the *entire* loaded dataset to a new CSV file.
        
        Parameters:
        -----------
        output_filepath : str
            Path for the output CSV file
        include_all_data : bool, default=True
            If True, includes all original columns. If False, only includes
            the target and predicted columns.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
            
        
        # Prepare the full dataset for prediction
        X_full = self.data.copy()
        
        if self.drop_cols:
            X_full = X_full.drop(columns=self.drop_cols, errors='ignore')
            
        X_full = X_full.drop(columns=[self.target_col], errors='ignore')
        
        # Ensure columns match training data
        X_full = X_full[self.feature_names]
        
        if self.scale_features and self.scaler is not None:
            X_full_scaled = pd.DataFrame(
                self.scaler.transform(X_full),
                columns=self.feature_names,
                index=X_full.index
            )
            predictions = self.model.predict(X_full_scaled)
        else:
            predictions = self.model.predict(X_full)
        
        if include_all_data:
            output_df = self.data.copy()
        else:
            output_df = pd.DataFrame()
            output_df[self.target_col] = self.data[self.target_col]
        
        output_df['predicted'] = predictions
        
        output_df.to_csv(output_filepath, index=False)
        print(f"Predictions exported to: {output_filepath}")
        
        return output_df
    
    def run_full_pipeline(self, input_filepath, output_filepath, include_all_data=True):
        """
        Run the complete pipeline: load, preprocess, train, evaluate, and export.
        
        Parameters:
        -----------
        input_filepath : str
            Path to input CSV file
        output_filepath : str
            Path for output CSV file with predictions
        include_all_data : bool, default=True
            Passed to export_predictions to control output file content.
            
        Returns:
        --------
        dict : Performance metrics
        """
        print("Starting full pipeline...\n")
        
        self.load_data(input_filepath)
        self.preprocess_data()
        self.train()
        metrics = self.evaluate()
        self.export_predictions(output_filepath, include_all_data=include_all_data)
        
        print("Pipeline completed successfully!")
        return metrics


# Example usage
if __name__ == "__main__":
    # Configure the model with all parameters during initialization
    clf = CardioRandomForestClassifier(
        test_size=0.2,
        random_state=42,
        n_estimators=200,
        max_depth=20,
        min_samples_split=30,
        min_samples_leaf=15,
        max_features='sqrt',
        class_weight='balanced',
        scale_features=True,
        target_col='cardio',  # <--- Parameter moved here
        drop_cols=['id']      # <--- Parameter moved here
    )
    
    # Run the pipeline with just the filepaths
    matrix_randomforest = clf.run_full_pipeline(
        input_filepath='train.csv',
        output_filepath='Prediction_RandomForest.csv' # Changed output name
    )


Starting full pipeline...

Data loaded successfully: 68520 rows, 13 columns
Data preprocessed: Train size = 54816, Test size = 13704
Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7310
Precision: 0.7448
Recall:    0.6933
F1-Score:  0.7181

Confusion Matrix:
[[5320 1609]
 [2078 4697]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.72      0.77      0.74      6929
     Disease       0.74      0.69      0.72      6775

    accuracy                           0.73     13704
   macro avg       0.73      0.73      0.73     13704
weighted avg       0.73      0.73      0.73     13704


Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7310
Precision: 0.7448
Recall:    0.6933
F1-Score:  0.7181

Confusion Matrix:
[[5320 1609]
 [2078 4697]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.72      0.77      0.74      6929
     Disease       0.74      0.69   

# Naive Base


In [11]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


class CardioNaiveBayesClassifier:
    """
    A reusable Naive Bayes classifier for cardiovascular disease prediction.
    
    Features:
    - Train/test split with customizable ratio
    - Optional feature scaling
    - Comprehensive performance metrics
    - Export predictions to CSV
    """
    
    def __init__(self, test_size=0.2, random_state=42, var_smoothing=1e-9, 
                 scale_features=True, target_col='cardio', drop_cols=None):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        test_size : float, default=0.2
            Proportion of dataset to include in test split
        random_state : int, default=42
            Random state for reproducibility
        var_smoothing : float, default=1e-9
            Portion of the largest variance of all features that is added to variances for calculation stability
        scale_features : bool, default=True
            Whether to scale features
        target_col : str, default='cardio'
            Name of the target variable column
        drop_cols : list, default=None
            List of columns to drop (e.g., ID columns)
        """

        self.test_size = test_size
        self.random_state = random_state
        self.var_smoothing = var_smoothing
        self.scale_features = scale_features
        self.target_col = target_col
        self.drop_cols = drop_cols if drop_cols else []
        
        self.model = None
        self.scaler = None
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None
        self.metrics = {}
        
    def load_data(self, filepath):
        """
        Load data from CSV file.
        
        Parameters:
        -----------
        filepath : str
            Path to the CSV file
            
        Returns:
        --------
        pd.DataFrame : Loaded dataframe
        """
        self.data = pd.read_csv(filepath)
        print(f"Data loaded successfully: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
        return self.data
    
    def preprocess_data(self):
        """
        Preprocess the data for training.
        Uses `self.target_col` and `self.drop_cols` from initialization.
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
        
        df = self.data.copy()
        if self.drop_cols:
            df = df.drop(columns=self.drop_cols, errors='ignore')
        
        X = df.drop(columns=[self.target_col])
        y = df[self.target_col]
        
        self.feature_names = X.columns.tolist()
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        if self.scale_features:
            self.scaler = StandardScaler()
            self.X_train = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                columns=self.feature_names,
                index=self.X_train.index
            )
            self.X_test = pd.DataFrame(
                self.scaler.transform(self.X_test),
                columns=self.feature_names,
                index=self.X_test.index
            )
        
        print(f"Data preprocessed: Train size = {len(self.X_train)}, Test size = {len(self.X_test)}")
    
    def train(self):
        """Train the Naive Bayes classifier."""
        if self.X_train is None:
            raise ValueError("Data not preprocessed. Call preprocess_data() first.")
        
        self.model = GaussianNB(
            var_smoothing=self.var_smoothing
        )
        
        self.model.fit(self.X_train, self.y_train)
        print("Model trained successfully!")
    
    def predict(self, X=None):
        """
        Make predictions on data.
        
        Parameters:
        -----------
        X : pd.DataFrame, default=None
            Data to predict. If None, uses test set.
            
        Returns:
        --------
        np.ndarray : Predictions
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        if X is None:
            X_to_predict = self.X_test
        else:
            X_to_predict = X

        if X is not None and self.scale_features and self.scaler is not None:
            # When predicting on new, external data
            X_to_predict = pd.DataFrame(
                self.scaler.transform(X_to_predict),
                columns=X_to_predict.columns,
                index=X_to_predict.index
            )
        
        return self.model.predict(X_to_predict)
    
    def evaluate(self):
        """
        Evaluate model performance on test set.
        
        Returns:
        --------
        dict : Dictionary containing all performance metrics
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        y_pred = self.predict()
        
        self.metrics = {
            'accuracy': accuracy_score(self.y_test, y_pred),
            'precision': precision_score(self.y_test, y_pred, average='binary'),
            'recall': recall_score(self.y_test, y_pred, average='binary'),
            'f1_score': f1_score(self.y_test, y_pred, average='binary'),
            'confusion_matrix': confusion_matrix(self.y_test, y_pred)
        }
        
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print(f"Accuracy:  {self.metrics['accuracy']:.4f}")
        print(f"Precision: {self.metrics['precision']:.4f}")
        print(f"Recall:    {self.metrics['recall']:.4f}")
        print(f"F1-Score:  {self.metrics['f1_score']:.4f}")
        print("\nConfusion Matrix:")
        print(self.metrics['confusion_matrix'])
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, 
                                      target_names=['No Disease', 'Disease']))
        print("="*50 + "\n")
        
        return self.metrics
    
    def export_predictions(self, output_filepath, include_all_data=True):
        """
        Export predictions for the *entire* loaded dataset to a new CSV file.
        
        Parameters:
        -----------
        output_filepath : str
            Path for the output CSV file
        include_all_data : bool, default=True
            If True, includes all original columns. If False, only includes
            the target and predicted columns.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
            
        
        # Prepare the full dataset for prediction
        X_full = self.data.copy()
        
        if self.drop_cols:
            X_full = X_full.drop(columns=self.drop_cols, errors='ignore')
            
        X_full = X_full.drop(columns=[self.target_col], errors='ignore')
        
        # Ensure columns match training data
        X_full = X_full[self.feature_names]
        
        if self.scale_features and self.scaler is not None:
            X_full_scaled = pd.DataFrame(
                self.scaler.transform(X_full),
                columns=self.feature_names,
                index=X_full.index
            )
            predictions = self.model.predict(X_full_scaled)
        else:
            predictions = self.model.predict(X_full)
        
        if include_all_data:
            output_df = self.data.copy()
        else:
            output_df = pd.DataFrame()
            output_df[self.target_col] = self.data[self.target_col]
        
        output_df['predicted'] = predictions
        
        output_df.to_csv(output_filepath, index=False)
        print(f"Predictions exported to: {output_filepath}")
        
        return output_df
    
    def run_full_pipeline(self, input_filepath, output_filepath, include_all_data=True):
        """
        Run the complete pipeline: load, preprocess, train, evaluate, and export.
        
        Parameters:
        -----------
        input_filepath : str
            Path to input CSV file
        output_filepath : str
            Path for output CSV file with predictions
        include_all_data : bool, default=True
            Passed to export_predictions to control output file content.
            
        Returns:
        --------
        dict : Performance metrics
        """
        print("Starting full pipeline...\n")
        
        self.load_data(input_filepath)
        self.preprocess_data()
        self.train()
        metrics = self.evaluate()
        self.export_predictions(output_filepath, include_all_data=include_all_data)
        
        print("Pipeline completed successfully!")
        return metrics


# Example usage
if __name__ == "__main__":
    # Configure the model with all parameters during initialization
    clf = CardioNaiveBayesClassifier(
        test_size=0.2,
        random_state=42,
        var_smoothing=1e-8,
        scale_features=True,
        target_col='cardio',  # <--- Parameter moved here
        drop_cols=['id']      # <--- Parameter moved here
    )
    
    # Run the pipeline with just the filepaths
    matrix_naivebayes = clf.run_full_pipeline(
        input_filepath='train.csv',
        output_filepath='Prediction_NaiveBayes.csv' # Changed output name
    )


Starting full pipeline...

Data loaded successfully: 68520 rows, 13 columns
Data preprocessed: Train size = 54816, Test size = 13704
Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7072
Precision: 0.7501
Recall:    0.6114
F1-Score:  0.6737

Confusion Matrix:
[[5549 1380]
 [2633 4142]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.68      0.80      0.73      6929
     Disease       0.75      0.61      0.67      6775

    accuracy                           0.71     13704
   macro avg       0.71      0.71      0.70     13704
weighted avg       0.71      0.71      0.70     13704


Predictions exported to: Prediction_NaiveBayes.csv
Pipeline completed successfully!
Predictions exported to: Prediction_NaiveBayes.csv
Pipeline completed successfully!


# Gradient Boosting


In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


class CardioGradientBoostingClassifier:
    """
    A reusable Gradient Boosting classifier for cardiovascular disease prediction.
    
    Features:
    - Train/test split with customizable ratio
    - Optional feature scaling
    - Comprehensive performance metrics
    - Export predictions to CSV
    """
    
    def __init__(self, test_size=0.2, random_state=42, n_estimators=100, 
                 learning_rate=0.1, max_depth=3, subsample=1.0,
                 scale_features=True, target_col='cardio', drop_cols=None):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        test_size : float, default=0.2
            Proportion of dataset to include in test split
        random_state : int, default=42
            Random state for reproducibility
        n_estimators : int, default=100
            The number of boosting stages to perform
        learning_rate : float, default=0.1
            Learning rate shrinks the contribution of each tree by learning_rate
        max_depth : int, default=3
            Maximum depth of the individual regression estimators
        subsample : float, default=1.0
            The fraction of samples to be used for fitting the individual base learners
        scale_features : bool, default=True
            Whether to scale features
        target_col : str, default='cardio'
            Name of the target variable column
        drop_cols : list, default=None
            List of columns to drop (e.g., ID columns)
        """

        self.test_size = test_size
        self.random_state = random_state
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.subsample = subsample
        self.scale_features = scale_features
        self.target_col = target_col
        self.drop_cols = drop_cols if drop_cols else []
        
        self.model = None
        self.scaler = None
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None
        self.metrics = {}
        
    def load_data(self, filepath):
        """
        Load data from CSV file.
        
        Parameters:
        -----------
        filepath : str
            Path to the CSV file
            
        Returns:
        --------
        pd.DataFrame : Loaded dataframe
        """
        self.data = pd.read_csv(filepath)
        print(f"Data loaded successfully: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
        return self.data
    
    def preprocess_data(self):
        """
        Preprocess the data for training.
        Uses `self.target_col` and `self.drop_cols` from initialization.
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
        
        df = self.data.copy()
        if self.drop_cols:
            df = df.drop(columns=self.drop_cols, errors='ignore')
        
        X = df.drop(columns=[self.target_col])
        y = df[self.target_col]
        
        self.feature_names = X.columns.tolist()
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        if self.scale_features:
            self.scaler = StandardScaler()
            self.X_train = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                columns=self.feature_names,
                index=self.X_train.index
            )
            self.X_test = pd.DataFrame(
                self.scaler.transform(self.X_test),
                columns=self.feature_names,
                index=self.X_test.index
            )
        
        print(f"Data preprocessed: Train size = {len(self.X_train)}, Test size = {len(self.X_test)}")
    
    def train(self):
        """Train the Gradient Boosting classifier."""
        if self.X_train is None:
            raise ValueError("Data not preprocessed. Call preprocess_data() first.")
        
        self.model = GradientBoostingClassifier(
            n_estimators=self.n_estimators,
            learning_rate=self.learning_rate,
            max_depth=self.max_depth,
            subsample=self.subsample,
            random_state=self.random_state
        )
        
        self.model.fit(self.X_train, self.y_train)
        print("Model trained successfully!")
    
    def predict(self, X=None):
        """
        Make predictions on data.
        
        Parameters:
        -----------
        X : pd.DataFrame, default=None
            Data to predict. If None, uses test set.
            
        Returns:
        --------
        np.ndarray : Predictions
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        if X is None:
            X_to_predict = self.X_test
        else:
            X_to_predict = X

        if X is not None and self.scale_features and self.scaler is not None:
            # When predicting on new, external data
            X_to_predict = pd.DataFrame(
                self.scaler.transform(X_to_predict),
                columns=X_to_predict.columns,
                index=X_to_predict.index
            )
        
        return self.model.predict(X_to_predict)
    
    def evaluate(self):
        """
        Evaluate model performance on test set.
        
        Returns:
        --------
        dict : Dictionary containing all performance metrics
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        y_pred = self.predict()
        
        self.metrics = {
            'accuracy': accuracy_score(self.y_test, y_pred),
            'precision': precision_score(self.y_test, y_pred, average='binary'),
            'recall': recall_score(self.y_test, y_pred, average='binary'),
            'f1_score': f1_score(self.y_test, y_pred, average='binary'),
            'confusion_matrix': confusion_matrix(self.y_test, y_pred)
        }
        
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print(f"Accuracy:  {self.metrics['accuracy']:.4f}")
        print(f"Precision: {self.metrics['precision']:.4f}")
        print(f"Recall:    {self.metrics['recall']:.4f}")
        print(f"F1-Score:  {self.metrics['f1_score']:.4f}")
        print("\nConfusion Matrix:")
        print(self.metrics['confusion_matrix'])
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, 
                                      target_names=['No Disease', 'Disease']))
        print("="*50 + "\n")
        
        return self.metrics
    
    def export_predictions(self, output_filepath, include_all_data=True):
        """
        Export predictions for the *entire* loaded dataset to a new CSV file.
        
        Parameters:
        -----------
        output_filepath : str
            Path for the output CSV file
        include_all_data : bool, default=True
            If True, includes all original columns. If False, only includes
            the target and predicted columns.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
            
        
        # Prepare the full dataset for prediction
        X_full = self.data.copy()
        
        if self.drop_cols:
            X_full = X_full.drop(columns=self.drop_cols, errors='ignore')
            
        X_full = X_full.drop(columns=[self.target_col], errors='ignore')
        
        # Ensure columns match training data
        X_full = X_full[self.feature_names]
        
        if self.scale_features and self.scaler is not None:
            X_full_scaled = pd.DataFrame(
                self.scaler.transform(X_full),
                columns=self.feature_names,
                index=X_full.index
            )
            predictions = self.model.predict(X_full_scaled)
        else:
            predictions = self.model.predict(X_full)
        
        if include_all_data:
            output_df = self.data.copy()
        else:
            output_df = pd.DataFrame()
            output_df[self.target_col] = self.data[self.target_col]
        
        output_df['predicted'] = predictions
        
        output_df.to_csv(output_filepath, index=False)
        print(f"Predictions exported to: {output_filepath}")
        
        return output_df
    
    def run_full_pipeline(self, input_filepath, output_filepath, include_all_data=True):
        """
        Run the complete pipeline: load, preprocess, train, evaluate, and export.
        
        Parameters:
        -----------
        input_filepath : str
            Path to input CSV file
        output_filepath : str
            Path for output CSV file with predictions
        include_all_data : bool, default=True
            Passed to export_predictions to control output file content.
            
        Returns:
        --------
        dict : Performance metrics
        """
        print("Starting full pipeline...\n")
        
        self.load_data(input_filepath)
        self.preprocess_data()
        self.train()
        metrics = self.evaluate()
        self.export_predictions(output_filepath, include_all_data=include_all_data)
        
        print("Pipeline completed successfully!")
        return metrics


# Example usage
if __name__ == "__main__":
    # Configure the model with all parameters during initialization
    clf = CardioGradientBoostingClassifier(
        test_size=0.2,
        random_state=42,
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        scale_features=True,
        target_col='cardio',  # <--- Parameter moved here
        drop_cols=['id']      # <--- Parameter moved here
    )
    
    # Run the pipeline with just the filepaths
    matrix_gradientboosting = clf.run_full_pipeline(
        input_filepath='train.csv',
        output_filepath='Prediction_GradientBoosting.csv' # Changed output name
    )


Starting full pipeline...

Data loaded successfully: 68520 rows, 13 columns
Data preprocessed: Train size = 54816, Test size = 13704
Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7330
Precision: 0.7514
Recall:    0.6874
F1-Score:  0.7180

Confusion Matrix:
[[5388 1541]
 [2118 4657]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.72      0.78      0.75      6929
     Disease       0.75      0.69      0.72      6775

    accuracy                           0.73     13704
   macro avg       0.73      0.73      0.73     13704
weighted avg       0.73      0.73      0.73     13704


Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7330
Precision: 0.7514
Recall:    0.6874
F1-Score:  0.7180

Confusion Matrix:
[[5388 1541]
 [2118 4657]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.72      0.78      0.75      6929
     Disease       0.75      0.69   

# SVM 


In [13]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


class CardioSupportVectorClassifier:
    """
    A reusable Support Vector Machine classifier for cardiovascular disease prediction.
    
    Features:
    - Train/test split with customizable ratio
    - Optional feature scaling
    - Comprehensive performance metrics
    - Export predictions to CSV
    """
    
    def __init__(self, test_size=0.2, random_state=42, C=1.0, 
                 kernel='rbf', gamma='scale', scale_features=True,
                 target_col='cardio', drop_cols=None):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        test_size : float, default=0.2
            Proportion of dataset to include in test split
        random_state : int, default=42
            Random state for reproducibility
        C : float, default=1.0
            Regularization parameter. The strength of the regularization is inversely proportional to C
        kernel : str, default='rbf'
            Specifies the kernel type to be used in the algorithm
        gamma : float or str, default='scale'
            Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
        scale_features : bool, default=True
            Whether to scale features
        target_col : str, default='cardio'
            Name of the target variable column
        drop_cols : list, default=None
            List of columns to drop (e.g., ID columns)
        """

        self.test_size = test_size
        self.random_state = random_state
        self.C = C
        self.kernel = kernel
        self.gamma = gamma
        self.scale_features = scale_features
        self.target_col = target_col
        self.drop_cols = drop_cols if drop_cols else []
        
        self.model = None
        self.scaler = None
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None
        self.metrics = {}
        
    def load_data(self, filepath):
        """
        Load data from CSV file.
        
        Parameters:
        -----------
        filepath : str
            Path to the CSV file
            
        Returns:
        --------
        pd.DataFrame : Loaded dataframe
        """
        self.data = pd.read_csv(filepath)
        print(f"Data loaded successfully: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
        return self.data
    
    def preprocess_data(self):
        """
        Preprocess the data for training.
        Uses `self.target_col` and `self.drop_cols` from initialization.
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
        
        df = self.data.copy()
        if self.drop_cols:
            df = df.drop(columns=self.drop_cols, errors='ignore')
        
        X = df.drop(columns=[self.target_col])
        y = df[self.target_col]
        
        self.feature_names = X.columns.tolist()
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        if self.scale_features:
            self.scaler = StandardScaler()
            self.X_train = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                columns=self.feature_names,
                index=self.X_train.index
            )
            self.X_test = pd.DataFrame(
                self.scaler.transform(self.X_test),
                columns=self.feature_names,
                index=self.X_test.index
            )
        
        print(f"Data preprocessed: Train size = {len(self.X_train)}, Test size = {len(self.X_test)}")
    
    def train(self):
        """Train the Support Vector Machine classifier."""
        if self.X_train is None:
            raise ValueError("Data not preprocessed. Call preprocess_data() first.")
        
        self.model = SVC(
            C=self.C,
            kernel=self.kernel,
            gamma=self.gamma,
            class_weight=self.class_weight,
            random_state=self.random_state
        )
        
        self.model.fit(self.X_train, self.y_train)
        print("Model trained successfully!")
    
    def predict(self, X=None):
        """
        Make predictions on data.
        
        Parameters:
        -----------
        X : pd.DataFrame, default=None
            Data to predict. If None, uses test set.
            
        Returns:
        --------
        np.ndarray : Predictions
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        if X is None:
            X_to_predict = self.X_test
        else:
            X_to_predict = X

        if X is not None and self.scale_features and self.scaler is not None:
            # When predicting on new, external data
            X_to_predict = pd.DataFrame(
                self.scaler.transform(X_to_predict),
                columns=X_to_predict.columns,
                index=X_to_predict.index
            )
        
        return self.model.predict(X_to_predict)
    
    def evaluate(self):
        """
        Evaluate model performance on test set.
        
        Returns:
        --------
        dict : Dictionary containing all performance metrics
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        y_pred = self.predict()
        
        self.metrics = {
            'accuracy': accuracy_score(self.y_test, y_pred),
            'precision': precision_score(self.y_test, y_pred, average='binary'),
            'recall': recall_score(self.y_test, y_pred, average='binary'),
            'f1_score': f1_score(self.y_test, y_pred, average='binary'),
            'confusion_matrix': confusion_matrix(self.y_test, y_pred)
        }
        
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print(f"Accuracy:  {self.metrics['accuracy']:.4f}")
        print(f"Precision: {self.metrics['precision']:.4f}")
        print(f"Recall:    {self.metrics['recall']:.4f}")
        print(f"F1-Score:  {self.metrics['f1_score']:.4f}")
        print("\nConfusion Matrix:")
        print(self.metrics['confusion_matrix'])
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, 
                                      target_names=['No Disease', 'Disease']))
        print("="*50 + "\n")
        
        return self.metrics
    
    def export_predictions(self, output_filepath, include_all_data=True):
        """
        Export predictions for the *entire* loaded dataset to a new CSV file.
        
        Parameters:
        -----------
        output_filepath : str
            Path for the output CSV file
        include_all_data : bool, default=True
            If True, includes all original columns. If False, only includes
            the target and predicted columns.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
            
        
        # Prepare the full dataset for prediction
        X_full = self.data.copy()
        
        if self.drop_cols:
            X_full = X_full.drop(columns=self.drop_cols, errors='ignore')
            
        X_full = X_full.drop(columns=[self.target_col], errors='ignore')
        
        # Ensure columns match training data
        X_full = X_full[self.feature_names]
        
        if self.scale_features and self.scaler is not None:
            X_full_scaled = pd.DataFrame(
                self.scaler.transform(X_full),
                columns=self.feature_names,
                index=X_full.index
            )
            predictions = self.model.predict(X_full_scaled)
        else:
            predictions = self.model.predict(X_full)
        
        if include_all_data:
            output_df = self.data.copy()
        else:
            output_df = pd.DataFrame()
            output_df[self.target_col] = self.data[self.target_col]
        
        output_df['predicted'] = predictions
        
        output_df.to_csv(output_filepath, index=False)
        print(f"Predictions exported to: {output_filepath}")
        
        return output_df
    
    def run_full_pipeline(self, input_filepath, output_filepath, include_all_data=True):
        """
        Run the complete pipeline: load, preprocess, train, evaluate, and export.
        
        Parameters:
        -----------
        input_filepath : str
            Path to input CSV file
        output_filepath : str
            Path for output CSV file with predictions
        include_all_data : bool, default=True
            Passed to export_predictions to control output file content.
            
        Returns:
        --------
        dict : Performance metrics
        """
        print("Starting full pipeline...\n")
        
        self.load_data(input_filepath)
        self.preprocess_data()
        self.train()
        metrics = self.evaluate()
        self.export_predictions(output_filepath, include_all_data=include_all_data)
        
        print("Pipeline completed successfully!")
        return metrics


# Example usage
if __name__ == "__main__":
    # Configure the model with all parameters during initialization
    clf = CardioSupportVectorClassifier(
        test_size=0.2,
        random_state=42,
        C=10.0,
        kernel='rbf',
        gamma='auto',
        class_weight='balanced',
        scale_features=True,
        target_col='cardio',  # <--- Parameter moved here
        drop_cols=['id']      # <--- Parameter moved here
    )
    
    # Run the pipeline with just the filepaths
    matrix_svm = clf.run_full_pipeline(
        input_filepath='train.csv',
        output_filepath='Prediction_SVM.csv' # Changed output name
    )


TypeError: CardioSupportVectorClassifier.__init__() got an unexpected keyword argument 'class_weight'

# KNN

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


class CardioKNeighborsClassifier:
    """
    A reusable K-Nearest Neighbors classifier for cardiovascular disease prediction.
    
    Features:
    - Train/test split with customizable ratio
    - Optional feature scaling
    - Comprehensive performance metrics
    - Export predictions to CSV
    """
    
    def __init__(self, test_size=0.2, random_state=42, n_neighbors=5, 
                 weights='uniform', metric='euclidean', scale_features=True,
                 target_col='cardio', drop_cols=None):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        test_size : float, default=0.2
            Proportion of dataset to include in test split
        random_state : int, default=42
            Random state for reproducibility
        n_neighbors : int, default=5
            Number of neighbors to use by default for kneighbors queries
        weights : str, default='uniform'
            Weight function used in prediction
        metric : str, default='euclidean'
            Distance metric to use for the tree
        scale_features : bool, default=True
            Whether to scale features
        target_col : str, default='cardio'
            Name of the target variable column
        drop_cols : list, default=None
            List of columns to drop (e.g., ID columns)
        """

        self.test_size = test_size
        self.random_state = random_state
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        self.scale_features = scale_features
        self.target_col = target_col
        self.drop_cols = drop_cols if drop_cols else []
        
        self.model = None
        self.scaler = None
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None
        self.metrics = {}
        
    def load_data(self, filepath):
        """
        Load data from CSV file.
        
        Parameters:
        -----------
        filepath : str
            Path to the CSV file
            
        Returns:
        --------
        pd.DataFrame : Loaded dataframe
        """
        self.data = pd.read_csv(filepath)
        print(f"Data loaded successfully: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
        return self.data
    
    def preprocess_data(self):
        """
        Preprocess the data for training.
        Uses `self.target_col` and `self.drop_cols` from initialization.
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
        
        df = self.data.copy()
        if self.drop_cols:
            df = df.drop(columns=self.drop_cols, errors='ignore')
        
        X = df.drop(columns=[self.target_col])
        y = df[self.target_col]
        
        self.feature_names = X.columns.tolist()
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        if self.scale_features:
            self.scaler = StandardScaler()
            self.X_train = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                columns=self.feature_names,
                index=self.X_train.index
            )
            self.X_test = pd.DataFrame(
                self.scaler.transform(self.X_test),
                columns=self.feature_names,
                index=self.X_test.index
            )
        
        print(f"Data preprocessed: Train size = {len(self.X_train)}, Test size = {len(self.X_test)}")
    
    def train(self):
        """Train the K-Nearest Neighbors classifier."""
        if self.X_train is None:
            raise ValueError("Data not preprocessed. Call preprocess_data() first.")
        
        self.model = KNeighborsClassifier(
            n_neighbors=self.n_neighbors,
            weights=self.weights,
            metric=self.metric
        )
        
        self.model.fit(self.X_train, self.y_train)
        print("Model trained successfully!")
    
    def predict(self, X=None):
        """
        Make predictions on data.
        
        Parameters:
        -----------
        X : pd.DataFrame, default=None
            Data to predict. If None, uses test set.
            
        Returns:
        --------
        np.ndarray : Predictions
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        if X is None:
            X_to_predict = self.X_test
        else:
            X_to_predict = X

        if X is not None and self.scale_features and self.scaler is not None:
            # When predicting on new, external data
            X_to_predict = pd.DataFrame(
                self.scaler.transform(X_to_predict),
                columns=X_to_predict.columns,
                index=X_to_predict.index
            )
        
        return self.model.predict(X_to_predict)
    
    def evaluate(self):
        """
        Evaluate model performance on test set.
        
        Returns:
        --------
        dict : Dictionary containing all performance metrics
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        y_pred = self.predict()
        
        self.metrics = {
            'accuracy': accuracy_score(self.y_test, y_pred),
            'precision': precision_score(self.y_test, y_pred, average='binary'),
            'recall': recall_score(self.y_test, y_pred, average='binary'),
            'f1_score': f1_score(self.y_test, y_pred, average='binary'),
            'confusion_matrix': confusion_matrix(self.y_test, y_pred)
        }
        
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print(f"Accuracy:  {self.metrics['accuracy']:.4f}")
        print(f"Precision: {self.metrics['precision']:.4f}")
        print(f"Recall:    {self.metrics['recall']:.4f}")
        print(f"F1-Score:  {self.metrics['f1_score']:.4f}")
        print("\nConfusion Matrix:")
        print(self.metrics['confusion_matrix'])
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, 
                                      target_names=['No Disease', 'Disease']))
        print("="*50 + "\n")
        
        return self.metrics
    
    def export_predictions(self, output_filepath, include_all_data=True):
        """
        Export predictions for the *entire* loaded dataset to a new CSV file.
        
        Parameters:
        -----------
        output_filepath : str
            Path for the output CSV file
        include_all_data : bool, default=True
            If True, includes all original columns. If False, only includes
            the target and predicted columns.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
            
        
        # Prepare the full dataset for prediction
        X_full = self.data.copy()
        
        if self.drop_cols:
            X_full = X_full.drop(columns=self.drop_cols, errors='ignore')
            
        X_full = X_full.drop(columns=[self.target_col], errors='ignore')
        
        # Ensure columns match training data
        X_full = X_full[self.feature_names]
        
        if self.scale_features and self.scaler is not None:
            X_full_scaled = pd.DataFrame(
                self.scaler.transform(X_full),
                columns=self.feature_names,
                index=X_full.index
            )
            predictions = self.model.predict(X_full_scaled)
        else:
            predictions = self.model.predict(X_full)
        
        if include_all_data:
            output_df = self.data.copy()
        else:
            output_df = pd.DataFrame()
            output_df[self.target_col] = self.data[self.target_col]
        
        output_df['predicted'] = predictions
        
        output_df.to_csv(output_filepath, index=False)
        print(f"Predictions exported to: {output_filepath}")
        
        return output_df
    
    def run_full_pipeline(self, input_filepath, output_filepath, include_all_data=True):
        """
        Run the complete pipeline: load, preprocess, train, evaluate, and export.
        
        Parameters:
        -----------
        input_filepath : str
            Path to input CSV file
        output_filepath : str
            Path for output CSV file with predictions
        include_all_data : bool, default=True
            Passed to export_predictions to control output file content.
            
        Returns:
        --------
        dict : Performance metrics
        """
        print("Starting full pipeline...\n")
        
        self.load_data(input_filepath)
        self.preprocess_data()
        self.train()
        metrics = self.evaluate()
        self.export_predictions(output_filepath, include_all_data=include_all_data)
        
        print("Pipeline completed successfully!")
        return metrics


# Example usage
if __name__ == "__main__":
    # Configure the model with all parameters during initialization
    clf = CardioKNeighborsClassifier(
        test_size=0.2,
        random_state=42,
        n_neighbors=15,
        weights='distance',
        metric='minkowski',
        scale_features=True,
        target_col='cardio',  # <--- Parameter moved here
        drop_cols=['id']      # <--- Parameter moved here
    )
    
    # Run the pipeline with just the filepaths
    matrix_knn = clf.run_full_pipeline(
        input_filepath='train.csv',
        output_filepath='Prediction_KNN.csv' # Changed output name
    )


Starting full pipeline...

Data loaded successfully: 68520 rows, 13 columns
Data preprocessed: Train size = 54816, Test size = 13704
Model trained successfully!
Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.6697
Precision: 0.6699
Recall:    0.6545
F1-Score:  0.6621

Confusion Matrix:
[[4744 2185]
 [2341 4434]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.67      0.68      0.68      6929
     Disease       0.67      0.65      0.66      6775

    accuracy                           0.67     13704
   macro avg       0.67      0.67      0.67     13704
weighted avg       0.67      0.67      0.67     13704



MODEL PERFORMANCE METRICS
Accuracy:  0.6697
Precision: 0.6699
Recall:    0.6545
F1-Score:  0.6621

Confusion Matrix:
[[4744 2185]
 [2341 4434]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.67      0.68      0.68      6929
     Disease       0.67      0.65   

# DL Algorithms


In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


class CardioMLPClassifier:
    """
    A reusable Multi-Layer Perceptron (MLP) deep learning classifier for cardiovascular disease prediction using PyTorch.
    
    Features:
    - Train/test split with customizable ratio
    - Optional feature scaling
    - Comprehensive performance metrics
    - Export predictions to CSV
    """
    
    def __init__(self, test_size=0.2, random_state=42, hidden_sizes=[64, 32], 
                 lr=0.001, epochs=100, batch_size=32, scale_features=True):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        test_size : float, default=0.2
            Proportion of dataset to include in test split
        random_state : int, default=42
            Random state for reproducibility
        hidden_sizes : list, default=[64, 32]
            List of hidden layer sizes
        lr : float, default=0.001
            Learning rate for the optimizer
        epochs : int, default=100
            Number of training epochs
        batch_size : int, default=32
            Batch size for training
        scale_features : bool, default=True
            Whether to scale features
        """

        self.test_size = test_size
        self.random_state = random_state
        self.hidden_sizes = hidden_sizes
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.scale_features = scale_features
        
        self.model = None
        self.scaler = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.X_train_tensor = None
        self.y_train_tensor = None
        self.feature_names = None
        self.metrics = {}
        self.drop_cols = None
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def load_data(self, filepath):
        """
        Load data from CSV file.
        
        Parameters:
        -----------
        filepath : str
            Path to the CSV file
            
        Returns:
        --------
        pd.DataFrame : Loaded dataframe
        """
        self.data = pd.read_csv(filepath)
        print(f"Data loaded successfully: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
        return self.data
    
    def preprocess_data(self, target_col='cardio', drop_cols=None):
        """
        Preprocess the data for training.
        
        Parameters:
        -----------
        target_col : str, default='cardio'
            Name of the target variable column
        drop_cols : list, default=None
            List of columns to drop (e.g., ID columns)
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
        
        self.drop_cols = drop_cols if drop_cols else []
        
        df = self.data.copy()
        if drop_cols:
            df = df.drop(columns=drop_cols, errors='ignore')
        
        X = df.drop(columns=[target_col])
        y = df[target_col]
        
        self.feature_names = X.columns.tolist()
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        if self.scale_features:
            self.scaler = StandardScaler()
            self.X_train = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                columns=self.feature_names,
                index=self.X_train.index
            )
            self.X_test = pd.DataFrame(
                self.scaler.transform(self.X_test),
                columns=self.feature_names,
                index=self.X_test.index
            )
        
        print(f"Data preprocessed: Train size = {len(self.X_train)}, Test size = {len(self.X_test)}")
    
    def train(self):
        """Train the MLP classifier."""
        if self.X_train is None:
            raise ValueError("Data not preprocessed. Call preprocess_data() first.")
        
        # Convert to tensors
        X_train_tensor = torch.FloatTensor(self.X_train.values).to(self.device)
        y_train_tensor = torch.LongTensor(self.y_train.values).to(self.device)
        
        self.X_train_tensor = X_train_tensor
        self.y_train_tensor = y_train_tensor
        
        # Define model
        input_size = len(self.feature_names)
        layers = []
        prev_size = input_size
        for size in self.hidden_sizes:
            layers.append(nn.Linear(prev_size, size))
            layers.append(nn.ReLU())
            prev_size = size
        layers.append(nn.Linear(prev_size, 2))  # Binary classification
        
        self.model = nn.Sequential(*layers).to(self.device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        
        # DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        
        self.model.train()
        for epoch in range(self.epochs):
            running_loss = 0.0
            for batch_x, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = self.model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
            if (epoch + 1) % 50 == 0:
                avg_loss = running_loss / len(train_loader)
                print(f'Epoch [{epoch+1}/{self.epochs}], Loss: {avg_loss:.4f}')
        
        print("Model trained successfully!")
    
    def predict(self, X=None):
        """
        Make predictions on data.
        
        Parameters:
        -----------
        X : pd.DataFrame, default=None
            Data to predict. If None, uses test set.
            
        Returns:
        --------
        np.ndarray : Predictions
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        self.model.eval()
        with torch.no_grad():
            if X is None:
                input_tensor = torch.FloatTensor(self.X_test.values).to(self.device)
            else:
                if self.scale_features and self.scaler is not None:
                    X_scaled = pd.DataFrame(
                        self.scaler.transform(X),
                        columns=X.columns,
                        index=X.index
                    )
                    input_tensor = torch.FloatTensor(X_scaled.values).to(self.device)
                else:
                    input_tensor = torch.FloatTensor(X.values).to(self.device)
            
            outputs = self.model(input_tensor)
            _, predicted = torch.max(outputs.data, 1)
            return predicted.cpu().numpy()
    
    def evaluate(self):
        """
        Evaluate model performance on test set.
        
        Returns:
        --------
        dict : Dictionary containing all performance metrics
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        y_pred = self.predict()
        
        self.metrics = {
            'accuracy': accuracy_score(self.y_test, y_pred),
            'precision': precision_score(self.y_test, y_pred, average='binary'),
            'recall': recall_score(self.y_test, y_pred, average='binary'),
            'f1_score': f1_score(self.y_test, y_pred, average='binary'),
            'confusion_matrix': confusion_matrix(self.y_test, y_pred)
        }
        
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print(f"Accuracy:  {self.metrics['accuracy']:.4f}")
        print(f"Precision: {self.metrics['precision']:.4f}")
        print(f"Recall:    {self.metrics['recall']:.4f}")
        print(f"F1-Score:  {self.metrics['f1_score']:.4f}")
        print("\nConfusion Matrix:")
        print(self.metrics['confusion_matrix'])
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, 
                                   target_names=['No Disease', 'Disease']))
        print("="*50 + "\n")
        
        return self.metrics
    
    def export_predictions(self, output_filepath, include_all_data=True):
        """
        Export predictions to a new CSV file.
        
        Parameters:
        -----------
        output_filepath : str
            Path for the output CSV file
        include_all_data : bool, default=True
            If True, includes all original columns. If False, only includes
            the target and predicted columns.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        X_full = self.data.copy()
        
        if self.drop_cols:
            X_full = X_full.drop(columns=self.drop_cols, errors='ignore')
        
        X_full = X_full.drop(columns=['cardio'], errors='ignore')
        
        if self.scale_features and self.scaler is not None:
            X_full_scaled = pd.DataFrame(
                self.scaler.transform(X_full),
                columns=self.feature_names,
                index=X_full.index
            )
            predictions = self.predict(X_full_scaled)
        else:
            predictions = self.predict(X_full)
        
        if include_all_data:
            output_df = self.data.copy()
        else:
            output_df = pd.DataFrame()
            output_df['cardio'] = self.data['cardio']
        
        output_df['predicted'] = predictions
        
        output_df.to_csv(output_filepath, index=False)
        print(f"Predictions exported to: {output_filepath}")
        
        return output_df
    
    def run_full_pipeline(self, input_filepath, output_filepath, 
                         target_col='cardio', drop_cols=None):
        """
        Run the complete pipeline: load, preprocess, train, evaluate, and export.
        
        Parameters:
        -----------
        input_filepath : str
            Path to input CSV file
        output_filepath : str
            Path for output CSV file with predictions
        target_col : str, default='cardio'
            Name of target variable
        drop_cols : list, default=None
            Columns to drop before training
            
        Returns:
        --------
        dict : Performance metrics
        """
        print("Starting full pipeline...\n")
        
        self.load_data(input_filepath)
        self.preprocess_data(target_col=target_col, drop_cols=drop_cols)
        self.train()
        metrics = self.evaluate()
        self.export_predictions(output_filepath)
        
        print("Pipeline completed successfully!")
        return metrics


class CardioTabularCNNClassifier:
    """
    A reusable 1D Convolutional Neural Network (CNN) classifier for tabular cardiovascular disease prediction using PyTorch.
    Treats features as a 1D sequence.
    
    Features:
    - Train/test split with customizable ratio
    - Optional feature scaling
    - Comprehensive performance metrics
    - Export predictions to CSV
    """
    
    def __init__(self, test_size=0.2, random_state=42, filters=[32, 64], 
                 kernel_sizes=[3, 3], lr=0.001, epochs=100, batch_size=32, 
                 scale_features=True):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        test_size : float, default=0.2
            Proportion of dataset to include in test split
        random_state : int, default=42
            Random state for reproducibility
        filters : list, default=[32, 64]
            List of number of filters for each conv layer
        kernel_sizes : list, default=[3, 3]
            List of kernel sizes for each conv layer
        lr : float, default=0.001
            Learning rate for the optimizer
        epochs : int, default=100
            Number of training epochs
        batch_size : int, default=32
            Batch size for training
        scale_features : bool, default=True
            Whether to scale features
        """

        self.test_size = test_size
        self.random_state = random_state
        self.filters = filters
        self.kernel_sizes = kernel_sizes
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.scale_features = scale_features
        
        self.model = None
        self.scaler = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.X_train_tensor = None
        self.y_train_tensor = None
        self.feature_names = None
        self.metrics = {}
        self.drop_cols = None
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def load_data(self, filepath):
        """
        Load data from CSV file.
        
        Parameters:
        -----------
        filepath : str
            Path to the CSV file
            
        Returns:
        --------
        pd.DataFrame : Loaded dataframe
        """
        self.data = pd.read_csv(filepath)
        print(f"Data loaded successfully: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
        return self.data
    
    def preprocess_data(self, target_col='cardio', drop_cols=None):
        """
        Preprocess the data for training.
        
        Parameters:
        -----------
        target_col : str, default='cardio'
            Name of the target variable column
        drop_cols : list, default=None
            List of columns to drop (e.g., ID columns)
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_data() first.")
        
        self.drop_cols = drop_cols if drop_cols else []
        
        df = self.data.copy()
        if drop_cols:
            df = df.drop(columns=drop_cols, errors='ignore')
        
        X = df.drop(columns=[target_col])
        y = df[target_col]
        
        self.feature_names = X.columns.tolist()
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        if self.scale_features:
            self.scaler = StandardScaler()
            self.X_train = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                columns=self.feature_names,
                index=self.X_train.index
            )
            self.X_test = pd.DataFrame(
                self.scaler.transform(self.X_test),
                columns=self.feature_names,
                index=self.X_test.index
            )
        
        print(f"Data preprocessed: Train size = {len(self.X_train)}, Test size = {len(self.X_test)}")
    
    def train(self):
        """Train the Tabular CNN classifier."""
        if self.X_train is None:
            raise ValueError("Data not preprocessed. Call preprocess_data() first.")
        
        # Convert to tensors with channel dimension for Conv1D
        X_train_tensor = torch.FloatTensor(self.X_train.values).unsqueeze(1).to(self.device)  # (N, 1, features)
        y_train_tensor = torch.LongTensor(self.y_train.values).to(self.device)
        
        self.X_train_tensor = X_train_tensor
        self.y_train_tensor = y_train_tensor
        
        # Define model
        input_size = len(self.feature_names)
        conv_layers = []
        prev_channels = 1
        prev_size = input_size
        for f, k in zip(self.filters, self.kernel_sizes):
            conv_layers.append(nn.Conv1d(prev_channels, f, kernel_size=k, padding=(k-1)//2))
            conv_layers.append(nn.ReLU())
            conv_layers.append(nn.MaxPool1d(2))
            prev_channels = f
            prev_size = prev_size // 2  # Approximate after pooling
        
        # Flatten size calculation (approximate)
        flatten_size = prev_channels * (input_size // 4)  # Assuming two poolings halve twice
        
        self.model = nn.Sequential(
            *conv_layers,
            nn.Flatten(),
            nn.Linear(flatten_size, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        ).to(self.device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        
        # DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        
        self.model.train()
        for epoch in range(self.epochs):
            running_loss = 0.0
            for batch_x, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = self.model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
            if (epoch + 1) % 50 == 0:
                avg_loss = running_loss / len(train_loader)
                print(f'Epoch [{epoch+1}/{self.epochs}], Loss: {avg_loss:.4f}')
        
        print("Model trained successfully!")
    
    def predict(self, X=None):
        """
        Make predictions on data.
        
        Parameters:
        -----------
        X : pd.DataFrame, default=None
            Data to predict. If None, uses test set.
            
        Returns:
        --------
        np.ndarray : Predictions
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        self.model.eval()
        with torch.no_grad():
            if X is None:
                input_tensor = torch.FloatTensor(self.X_test.values).unsqueeze(1).to(self.device)
            else:
                if self.scale_features and self.scaler is not None:
                    X_scaled = pd.DataFrame(
                        self.scaler.transform(X),
                        columns=X.columns,
                        index=X.index
                    )
                    input_tensor = torch.FloatTensor(X_scaled.values).unsqueeze(1).to(self.device)
                else:
                    input_tensor = torch.FloatTensor(X.values).unsqueeze(1).to(self.device)
            
            outputs = self.model(input_tensor)
            _, predicted = torch.max(outputs.data, 1)
            return predicted.cpu().numpy()
    
    def evaluate(self):
        """
        Evaluate model performance on test set.
        
        Returns:
        --------
        dict : Dictionary containing all performance metrics
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        y_pred = self.predict()
        
        self.metrics = {
            'accuracy': accuracy_score(self.y_test, y_pred),
            'precision': precision_score(self.y_test, y_pred, average='binary'),
            'recall': recall_score(self.y_test, y_pred, average='binary'),
            'f1_score': f1_score(self.y_test, y_pred, average='binary'),
            'confusion_matrix': confusion_matrix(self.y_test, y_pred)
        }
        
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print(f"Accuracy:  {self.metrics['accuracy']:.4f}")
        print(f"Precision: {self.metrics['precision']:.4f}")
        print(f"Recall:    {self.metrics['recall']:.4f}")
        print(f"F1-Score:  {self.metrics['f1_score']:.4f}")
        print("\nConfusion Matrix:")
        print(self.metrics['confusion_matrix'])
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, 
                                   target_names=['No Disease', 'Disease']))
        print("="*50 + "\n")
        
        return self.metrics
    
    def export_predictions(self, output_filepath, include_all_data=True):
        """
        Export predictions to a new CSV file.
        
        Parameters:
        -----------
        output_filepath : str
            Path for the output CSV file
        include_all_data : bool, default=True
            If True, includes all original columns. If False, only includes
            the target and predicted columns.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        X_full = self.data.copy()
        
        if self.drop_cols:
            X_full = X_full.drop(columns=self.drop_cols, errors='ignore')
        
        X_full = X_full.drop(columns=['cardio'], errors='ignore')
        
        if self.scale_features and self.scaler is not None:
            X_full_scaled = pd.DataFrame(
                self.scaler.transform(X_full),
                columns=self.feature_names,
                index=X_full.index
            )
            predictions = self.predict(X_full_scaled)
        else:
            predictions = self.predict(X_full)
        
        if include_all_data:
            output_df = self.data.copy()
        else:
            output_df = pd.DataFrame()
            output_df['cardio'] = self.data['cardio']
        
        output_df['predicted'] = predictions
        
        output_df.to_csv(output_filepath, index=False)
        print(f"Predictions exported to: {output_filepath}")
        
        return output_df
    
    def run_full_pipeline(self, input_filepath, output_filepath, 
                         target_col='cardio', drop_cols=None):
        """
        Run the complete pipeline: load, preprocess, train, evaluate, and export.
        
        Parameters:
        -----------
        input_filepath : str
            Path to input CSV file
        output_filepath : str
            Path for output CSV file with predictions
        target_col : str, default='cardio'
            Name of target variable
        drop_cols : list, default=None
            Columns to drop before training
            
        Returns:
        --------
        dict : Performance metrics
        """
        print("Starting full pipeline...\n")
        
        self.load_data(input_filepath)
        self.preprocess_data(target_col=target_col, drop_cols=drop_cols)
        self.train()
        metrics = self.evaluate()
        self.export_predictions(output_filepath)
        
        print("Pipeline completed successfully!")
        return metrics


# Example usage
if __name__ == "__main__":
    # MLP Example
    print("Running MLP Pipeline...")
    mlp_clf = CardioMLPClassifier(
        test_size=0.2,
        random_state=42,
        hidden_sizes=[128, 64, 32],
        lr=0.0005,
        epochs=150,
        batch_size=64,
        scale_features=True
    )
    
    matrix_mlp = mlp_clf.run_full_pipeline(
        input_filepath='train.csv',
        output_filepath='Prediction_MLP.csv',
        drop_cols=['id'] 
    )
    
    # CNN Example
    # print("\nRunning Tabular CNN Pipeline...")
    # cnn_clf = CardioTabularCNNClassifier(
    #     test_size=0.2,
    #     random_state=42,
    #     filters=[32, 64],
    #     kernel_sizes=[3, 3],
    #     lr=0.001,
    #     epochs=100,
    #     batch_size=32,
    #     scale_features=True
    # )
    
    # cnn_metrics = cnn_clf.run_full_pipeline(
    #     input_filepath='cardio_train.csv',
    #     output_filepath='cardio_train_cnn_pred.csv',
    #     drop_cols=['id'] 
    # )

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.
Running MLP Pipeline...
Starting full pipeline...

Data loaded successfully: 68520 rows, 13 columns
Data preprocessed: Train size = 54816, Test size = 13704


ERROR: Could not find a version that satisfies the requirement torchaudio (from versions: none)
ERROR: No matching distribution found for torchaudio


Epoch [50/100], Loss: 0.5279
Epoch [100/100], Loss: 0.5191
Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7221
Precision: 0.7469
Recall:    0.6624
F1-Score:  0.7021

Confusion Matrix:
[[5408 1521]
 [2287 4488]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.70      0.78      0.74      6929
     Disease       0.75      0.66      0.70      6775

    accuracy                           0.72     13704
   macro avg       0.72      0.72      0.72     13704
weighted avg       0.72      0.72      0.72     13704


Epoch [100/100], Loss: 0.5191
Model trained successfully!

MODEL PERFORMANCE METRICS
Accuracy:  0.7221
Precision: 0.7469
Recall:    0.6624
F1-Score:  0.7021

Confusion Matrix:
[[5408 1521]
 [2287 4488]]

Classification Report:
              precision    recall  f1-score   support

  No Disease       0.70      0.78      0.74      6929
     Disease       0.75      0.66      0.70      6775

    accuracy             

# Final Voted Ensamble algorithm


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# Model configuration
models_config = {
    'LogisticRegression': {
        'file': 'Prediction_LogisticRegression.csv',
        'metrics': matrix_logisticregression
    },
    'RandomForest': {
        'file': 'Prediction_RandomForest.csv',
        'metrics': matrix_randomforest
    },
    'NaiveBayes': {
        'file': 'Prediction_NaiveBayes.csv',
        'metrics': matrix_naivebayes
    },
    'GradientBoosting': {
        'file': 'Prediction_GradientBoosting.csv',
        'metrics': matrix_gradientboosting
    },
    'SVM': {
        'file': 'Prediction_SVM.csv',
        'metrics': matrix_svm
    },
    'KNN': {
        'file': 'Prediction_KNN.csv',
        'metrics': matrix_knn
    },
    'MLP': {
        'file': 'Prediction_MLP.csv',
        'metrics': matrix_mlp
    }
}
print(models_config)
def load_predictions():
    """Load all prediction files and extract predicted column"""
    predictions_dict = {}
    base_df = None
    
    for model_name, config in models_config.items():
        try:
            df = pd.read_csv(config['file'])
            
            # Store base dataframe (all columns except predicted)
            if base_df is None:
                base_df = df.drop(columns=['predicted']) if 'predicted' in df.columns else df.copy()
            
            # Extract predictions
            if 'predicted' in df.columns:
                predictions_dict[model_name] = df['predicted'].values
            else:
                print(f"Warning: 'predicted' column not found in {config['file']}")
                
        except FileNotFoundError:
            print(f"Warning: File {config['file']} not found. Skipping {model_name}.")
        except Exception as e:
            print(f"Error loading {config['file']}: {str(e)}")
    
    return base_df, predictions_dict

def weighted_voting(predictions_dict, models_config):
    """
    Perform weighted voting based on model accuracies
    
    Args:
        predictions_dict: Dictionary of model predictions
        models_config: Configuration with accuracy metrics
    
    Returns:
        final_predictions: Array of final predictions
        weights: Dictionary of normalized weights
    """
    # Extract accuracies and create weights
    weights = {}
    for model_name in predictions_dict.keys():
        accuracy = models_config[model_name]['metrics']['accuracy']
        weights[model_name] = accuracy
    
    # Normalize weights to sum to 1
    total_weight = sum(weights.values())
    normalized_weights = {k: v/total_weight for k, v in weights.items()}
    
    print("\n=== Model Weights (Based on Accuracy) ===")
    for model, weight in sorted(normalized_weights.items(), key=lambda x: x[1], reverse=True):
        print(f"{model}: {weight:.4f} (Accuracy: {models_config[model]['metrics']['accuracy']:.4f})")
    
    # Perform weighted voting
    n_samples = len(next(iter(predictions_dict.values())))
    weighted_votes = np.zeros(n_samples)
    
    for model_name, predictions in predictions_dict.items():
        weight = normalized_weights[model_name]
        weighted_votes += predictions * weight
    
    # Final prediction: if weighted vote >= 0.5, predict 1, else 0
    final_predictions = (weighted_votes >= 0.5).astype(int)
    
    return final_predictions, normalized_weights

def calculate_metrics(y_true, y_pred):
    """Calculate comprehensive metrics"""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    cm = confusion_matrix(y_true, y_pred)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm
    }

def main():
    print("=" * 60)
    print("WEIGHTED ENSEMBLE VOTING FOR CARDIO PREDICTION")
    print("=" * 60)
    
    # Load all predictions
    print("\n1. Loading prediction files...")
    base_df, predictions_dict = load_predictions()
    
    if not predictions_dict:
        print("Error: No prediction files loaded successfully.")
        return
    
    print(f"Successfully loaded {len(predictions_dict)} model predictions")
    print(f"Number of samples: {len(next(iter(predictions_dict.values())))}")
    
    # Perform weighted voting
    print("\n2. Performing weighted ensemble voting...")
    final_predictions, weights = weighted_voting(predictions_dict, models_config)
    
    # Add final predictions to dataframe
    final_df = base_df.copy()
    final_df['predicted'] = final_predictions
    
    # Calculate ensemble metrics if ground truth is available
    if 'cardio' in final_df.columns:
        print("\n3. Calculating ensemble metrics...")
        ensemble_metrics = calculate_metrics(final_df['cardio'], final_predictions)
        
        print("\n=== Ensemble Model Performance ===")
        print(f"Accuracy:  {ensemble_metrics['accuracy']:.4f}")
        print(f"Precision: {ensemble_metrics['precision']:.4f}")
        print(f"Recall:    {ensemble_metrics['recall']:.4f}")
        print(f"F1-Score:  {ensemble_metrics['f1_score']:.4f}")
        print(f"\nConfusion Matrix:")
        print(ensemble_metrics['confusion_matrix'])
        print(f"  TN: {ensemble_metrics['confusion_matrix'][0,0]}, FP: {ensemble_metrics['confusion_matrix'][0,1]}")
        print(f"  FN: {ensemble_metrics['confusion_matrix'][1,0]}, TP: {ensemble_metrics['confusion_matrix'][1,1]}")
        
        # Compare with individual models
        print("\n=== Comparison with Individual Models ===")
        comparison_data = []
        for model_name, config in models_config.items():
            if model_name in predictions_dict:
                comparison_data.append({
                    'Model': model_name,
                    'Accuracy': config['metrics']['accuracy'],
                    'Precision': config['metrics']['precision'],
                    'Recall': config['metrics']['recall'],
                    'F1-Score': config['metrics']['f1_score']
                })
        
        comparison_data.append({
            'Model': '*** ENSEMBLE ***',
            'Accuracy': ensemble_metrics['accuracy'],
            'Precision': ensemble_metrics['precision'],
            'Recall': ensemble_metrics['recall'],
            'F1-Score': ensemble_metrics['f1_score']
        })
        
        comparison_df = pd.DataFrame(comparison_data)
        print(comparison_df.to_string(index=False))
        
        # Check if ensemble improved
        max_individual_accuracy = max([config['metrics']['accuracy'] for config in models_config.values()])
        improvement = ensemble_metrics['accuracy'] - max_individual_accuracy
        print(f"\nImprovement over best individual model: {improvement:+.4f}")
    
    # Save final predictions
    output_file = 'Prediction_Ensemble_Weighted.csv'
    final_df.to_csv(output_file, index=False)
    print(f"\n4. Final predictions saved to: {output_file}")
    
    # Summary statistics
    print("\n=== Prediction Summary ===")
    print(f"Total predictions: {len(final_predictions)}")
    print(f"predicted cardio=0: {np.sum(final_predictions == 0)} ({np.sum(final_predictions == 0)/len(final_predictions)*100:.2f}%)")
    print(f"predicted cardio=1: {np.sum(final_predictions == 1)} ({np.sum(final_predictions == 1)/len(final_predictions)*100:.2f}%)")
    
    print("\n" + "=" * 60)
    print("ENSEMBLE VOTING COMPLETED SUCCESSFULLY")
    print("=" * 60)

if __name__ == "__main__":
    main()

{'LogisticRegression': {'file': 'Prediction_LogisticRegression.csv', 'metrics': {'accuracy': 0.7249951352403191, 'precision': 0.7508342602892102, 'recall': 0.6641739643805963, 'f1_score': 0.7048504150785778, 'confusion_matrix': array([[8153, 2240],
       [3413, 6750]])}}, 'RandomForest': {'file': 'Prediction_RandomForest.csv', 'metrics': {'accuracy': 0.7298598949211909, 'precision': 0.7464314354450682, 'recall': 0.6869372693726937, 'f1_score': 0.7154496541122214, 'confusion_matrix': array([[5348, 1581],
       [2121, 4654]])}}, 'NaiveBayes': {'file': 'Prediction_NaiveBayes.csv', 'metrics': {'accuracy': 0.7071657910099242, 'precision': 0.7500905469032959, 'recall': 0.6113653136531365, 'f1_score': 0.673660242335529, 'confusion_matrix': array([[5549, 1380],
       [2633, 4142]])}}, 'GradientBoosting': {'file': 'Prediction_GradientBoosting.csv', 'metrics': {'accuracy': 0.7247518972562755, 'precision': 0.7400479616306954, 'recall': 0.6832472324723248, 'f1_score': 0.7105141980046048, 'confu